In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium 
from folium.features import DivIcon
import branca.colormap as cm

Si leggono i file csv

In [2]:
wdi=pd.read_csv('WDI_csv\WDIData.csv')
country=pd.read_csv('WDI_csv\WDICountry.csv')
world_url=r'Geospatial_data\Longitude_Graticules_and_World_Countries_Boundaries-shp\99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp'

Si ottengono tutti gli stati appartenenti all'Africa Sub-Sahariana

In [3]:
africa=country[country['Region'].isin(['Sub-Saharan Africa','Middle East & North Africa'])]['Short Name']

Si eliminano gli stati del Middle East

In [4]:
africa.loc[[8,22,111,112,114,117,126,129,158,181,195,199,204,226,261]];

In [5]:
africa=africa.drop([8,22,111,112,114,117,126,129,158,181,195,199,204,226,261])

In [6]:
africa=africa.sort_values()

In [7]:
np.array(africa);

Alcuni stati sono scritti in maniera differente nei due dataset, andiamo a modificare i nomi contenuti in africa in modo da renderli concordi a quelli contenuti nella colonna 'Country Name' del dataset wdi

In [8]:
africa.replace({"Côte d'Ivoire":"Cote d'Ivoire"}, inplace=True)
africa.replace({'Dem. Rep. Congo':'Congo, Dem. Rep.'}, inplace=True)
africa.replace({"Congo":"Congo, Rep."}, inplace=True)
africa.replace({"Egypt":"Egypt, Arab Rep."}, inplace=True)
africa.replace({"São Tomé and Principe":"Sao Tome and Principe"}, inplace=True)
africa.replace({"The Gambia":"Gambia, The"}, inplace=True)

Si filtra il dataset wdi mantenendo solo le righe associate agli stati dell'Africa Sub-Sahariana

In [9]:
wdi_africa=wdi[wdi['Country Name'].isin(africa)]

Aggiungiamo ora il Western Sahara all'elenco degli stati, non essendo presente

In [10]:
africa.loc[len(africa)]='W. Sahara'
africa=africa.sort_values()
africa=africa.reset_index(drop=True)

Si definisce la lista i, contenente gli indici di interesse e che si vuole visualizzare

In [11]:
i1='People using at least basic drinking water services (% of population)'
i2='People using at least basic drinking water services, rural (% of rural population)'
i3='People using at least basic drinking water services, urban (% of urban population)'
i_name=[i1,i2,i3]

Si filtra il dataset wdi_africa, mantenendo solo le righe associatre agli indici contenuti nella lista i

In [12]:
wdi_africa_water=wdi_africa[wdi_africa['Indicator Name'].isin(i_name)]

Si filtra nuovamente il dataset mantenendo solo le colonne dall'anno 2000 in poi 

In [13]:
wdi_new=wdi_africa_water
year_of_interest=[]
for year in wdi.columns[44:65]:
    year_of_interest.append(year)
wdi_africa_water=wdi_new[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']+year_of_interest]

Si creano 3 nuovi datase, ottenuti da wdi_africa_water distinguendo gli indici (total, rural ed urban), che vengono riorganizzati opportunamente

In [14]:
total=wdi_africa_water[wdi_africa_water['Indicator Name']==i1]
rural=wdi_africa_water[wdi_africa_water['Indicator Name']==i2]
urban=wdi_africa_water[wdi_africa_water['Indicator Name']==i3]


new_row=[np.nan]*len(total.iloc[0])
new_row[0]='W. Sahara'
total.loc[0]=new_row
rural.loc[0]=new_row
urban.loc[0]=new_row

total=total.sort_values('Country Name')
rural=rural.sort_values('Country Name')
urban=urban.sort_values('Country Name')

total=total.reset_index(drop=True)
rural=rural.reset_index(drop=True)
urban=urban.reset_index(drop=True)

index_type=[total,rural,urban]

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Si creano le colormap per le due mappe

In [15]:
colormap_1=cm.linear.YlGn_09.scale(0,100).to_step(5)
colormap_2=cm.LinearColormap(colors=['mistyrose','red'], index=[0,80],vmin=0,vmax=80).to_step(5)
colormap_3=cm.LinearColormap(colors=['blue','lightblue'], index=[-40,0],vmin=-40,vmax=0).to_step(2)

Si definisce una variabile names contenente le 3 tipologie di popolazione considerata

In [16]:
names=['% of population','% of rural population','% of urban population']

Dataset relativo all'indice totale

In [17]:
wdi_africa_water_total=wdi_africa_water[wdi_africa_water['Indicator Name']==i1]
wdi_africa_water_total=wdi_africa_water_total.transpose()
wdi_africa_water_total.columns=wdi_africa_water_total.iloc[0]
#si aggiunge una colonna di nan per il Western Sahara che non è presente nel dataset
wdi_africa_water_total['W. Sahara']=[np.nan]*len(wdi_africa_water_total['Algeria'])
#riordiniamo le colonne in senso crescente
wdi_africa_water_total=wdi_africa_water_total[wdi_africa_water_total.columns.sort_values()]
wdi_africa_water_total['year']=wdi_africa_water_total.index
wdi_africa_water_total.set_index([pd.Index(list(range(len(wdi_africa_water_total))))], inplace=True)

Si eliminano le colonne non necessarie

In [18]:
wdi_africa_water_total=wdi_africa_water_total.iloc[4:]

Si converte la colonna 'year' in un formato data in nanosecondi e poi nuovamente in stringa

In [19]:
wdi_africa_water_total['year_int']=wdi_africa_water_total['year'].apply(str)
wdi_africa_water_total['year_int']=pd.to_datetime(wdi_africa_water_total['year_int']).astype(np.int64) / 10**9
wdi_africa_water_total['year_int']=wdi_africa_water_total['year_int'].astype(np.int64)

C:\Users\Marco\AppData\Local\Temp/ipykernel_23196/1795029705.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  wdi_africa_water_total['year_int']=pd.to_datetime(wdi_africa_water_total['year_int']).astype(np.int64) / 10**9


Siccome ci sono alcuni valori nulli e colormap associerebbe di default a questi uno dei due colori all'estremo della colormap stessa, si definisce una nuova funzione che associa il grigio ai valori nulli

In [20]:
def color_map_1(x):
    if pd.isna(x):
        return('#808080')
    else:
        return(colormap_1(x))


def color_map_2(x):
    if pd.isna(x):
        return('#808080')
    elif x>0:
        return(colormap_2(x))
    elif x<=0:
        return(colormap_3(x))

Si aggiunge una colonna per ogni stato con il colore della colormap associato alla percentuale di popolazione

In [21]:
for country in africa:
    wdi_africa_water_total[country+'-color']=wdi_africa_water_total[country].apply(color_map_1)
    

Si crea lo style dictionary da usare per il TimeSliderChoropleth

In [22]:
country_list=list(africa)
country_idx=range(len(country_list))

style_dict_total={}
for i in country_idx:
    country = country_list[i]
    r = wdi_africa_water_total[[country+'-color','year_int']]
    inner_dict = {}
    for j in range(len(r)):
        inner_dict[r['year_int'].to_list()[j]] = {'color': r[country+'-color'].to_list()[j], 'opacity': 0.7}
    style_dict_total[str(i)] = inner_dict

Dataset relativo all'indice Rurale

In [23]:
wdi_africa_water_rural=wdi_africa_water[wdi_africa_water['Indicator Name']==i2]
wdi_africa_water_rural=wdi_africa_water_rural.transpose()
wdi_africa_water_rural.columns=wdi_africa_water_rural.iloc[0]
#si aggiunge una colonna di nan per il Western Sahara che non è presente nel dataset
wdi_africa_water_rural['W. Sahara']=[np.nan]*len(wdi_africa_water_rural['Algeria'])
#riordiniamo le colonne in senso crescente
wdi_africa_water_total=wdi_africa_water_total[wdi_africa_water_total.columns.sort_values()]
wdi_africa_water_rural['year']=wdi_africa_water_rural.index
wdi_africa_water_rural.set_index([pd.Index(list(range(len(wdi_africa_water_rural))))], inplace=True)

In [24]:
wdi_africa_water_rural=wdi_africa_water_rural.iloc[4:]

Si aggiunge una colonna per ogni stato con il colore della colormap associato alla percentuale di popolazione

In [25]:
for country in africa:
    wdi_africa_water_rural[country+'-color']=wdi_africa_water_rural[country].apply(color_map_1)

Si converte la colonna 'year' in un formato data in nanosecondi

In [26]:
wdi_africa_water_rural['year_int']=wdi_africa_water_rural['year'].apply(str)
wdi_africa_water_rural['year_int']=pd.to_datetime(wdi_africa_water_rural['year_int']).astype(np.int64) / 10**9
wdi_africa_water_rural['year_int']=wdi_africa_water_rural['year_int'].astype(np.int64)

C:\Users\Marco\AppData\Local\Temp/ipykernel_23196/663627941.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  wdi_africa_water_rural['year_int']=pd.to_datetime(wdi_africa_water_rural['year_int']).astype(np.int64) / 10**9


Si crea lo style dictionary da usare per il TimeSliderChoropleth

In [27]:
country_list=list(africa)
country_idx=range(len(country_list))

style_dict_rural={}
for i in country_idx:
    country = country_list[i]
    r = wdi_africa_water_rural[[country+'-color','year_int']]
    inner_dict = {}
    for j in range(len(r)):
        inner_dict[r['year_int'].to_list()[j]] = {'color': r[country+'-color'].to_list()[j], 'opacity': 0.7}
    style_dict_rural[str(i)] = inner_dict

Dataset relativo all'indice Urbano

In [28]:
wdi_africa_water_urban=wdi_africa_water[wdi_africa_water['Indicator Name']==i3]
wdi_africa_water_urban=wdi_africa_water_urban.transpose()
wdi_africa_water_urban.columns=wdi_africa_water_urban.iloc[0]
#si aggiunge una colonna di nan per il Western Sahara che non è presente nel dataset
wdi_africa_water_urban['W. Sahara']=[np.nan]*len(wdi_africa_water_urban['Algeria'])
#riordiniamo le colonne in senso crescente
wdi_africa_water_total=wdi_africa_water_total[wdi_africa_water_total.columns.sort_values()]
wdi_africa_water_urban['year']=wdi_africa_water_urban.index
wdi_africa_water_urban.set_index([pd.Index(list(range(len(wdi_africa_water_urban))))], inplace=True)

In [29]:
wdi_africa_water_urban=wdi_africa_water_urban.iloc[4:]

Si aggiunge una colonna per ogni stato con il colore della colormap associato alla percentuale di popolazione

In [30]:
for country in africa:
    wdi_africa_water_urban[country+'-color']=wdi_africa_water_urban[country].apply(color_map_1)

Si converte la colonna 'year' in un formato data in nanosecondi

In [31]:
wdi_africa_water_urban['year_int']=wdi_africa_water_urban['year'].apply(str)
wdi_africa_water_urban['year_int']=pd.to_datetime(wdi_africa_water_urban['year_int']).astype(np.int64) / 10**9
wdi_africa_water_urban['year_int']=wdi_africa_water_urban['year_int'].astype(np.int64)

C:\Users\Marco\AppData\Local\Temp/ipykernel_23196/1904111790.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  wdi_africa_water_urban['year_int']=pd.to_datetime(wdi_africa_water_urban['year_int']).astype(np.int64) / 10**9


Si aggiunge una colonna per ogni stato con il colore della colormap associato alla percentuale di popolazione

In [32]:
country_list=list(africa)
country_idx=range(len(country_list))

style_dict_urban={}
for i in country_idx:
    country = country_list[i]
    r = wdi_africa_water_urban[[country+'-color','year_int']]
    inner_dict = {}
    for j in range(len(r)):
        inner_dict[r['year_int'].to_list()[j]] = {'color': r[country+'-color'].to_list()[j], 'opacity': 0.7}
    style_dict_urban[str(i)] = inner_dict

Si crea ora lo style dict associato alla differenza tra percentuale urbana e rurale

In [33]:
wdi_africa_water_difference=wdi_africa_water_urban[list(africa)]-wdi_africa_water_rural[list(africa)]

Si aggiunge una colonna per ogni stato con il colore della colormap associato alla percentuale di popolazione

In [34]:
for country in africa:
    wdi_africa_water_difference[country+'-color']=wdi_africa_water_difference[country].apply(color_map_2)

Aggiungiamo una colonna relativa all'anno

In [35]:
wdi_africa_water_difference['year']=year_of_interest

Si converte la colonna 'year' in un formato data in nanosecondi

In [36]:
wdi_africa_water_difference['year_int']=wdi_africa_water_difference['year'].apply(str)
wdi_africa_water_difference['year_int']=pd.to_datetime(wdi_africa_water_difference['year_int']).astype(np.int64) / 10**9
wdi_africa_water_difference['year_int']=wdi_africa_water_difference['year_int'].astype(np.int64)

C:\Users\Marco\AppData\Local\Temp/ipykernel_23196/2868599074.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  wdi_africa_water_difference['year_int']=pd.to_datetime(wdi_africa_water_difference['year_int']).astype(np.int64) / 10**9


In [37]:
country_list=list(africa)
country_idx=range(len(country_list))

style_dict_difference={}
for i in country_idx:
    country = country_list[i]
    r = wdi_africa_water_difference[[country+'-color','year_int']]
    inner_dict = {}
    for j in range(len(r)):
        inner_dict[r['year_int'].to_list()[j]] = {'color': r[country+'-color'].to_list()[j], 'opacity': 0.7}
    style_dict_difference[str(i+55)] = inner_dict

In [38]:
style_dict=[style_dict_total,style_dict_difference,style_dict_rural,style_dict_urban]
wdi_africa_water_=[wdi_africa_water_total,wdi_africa_water_difference,wdi_africa_water_rural,wdi_africa_water_urban]

Si definisce una lista coi possibili sfondi da applicare alla mappa

In [62]:
tiles=['OpenStreetMap',None, 'Stamen Toner','stamenwatercolor',
       'Stamen Terrain','cartodbdark_matter',
       'cartodbpositron','cartodbpositronnolabels']

Si legge ora lo shapefile necessario contenente la geometria degli stati

In [63]:
world_shp=gpd.read_file(world_url)

Si osserva che mentre nel dataset Sudan e South Sudan sono distinti, nello shape file no, dobbiamo quindi modificarlo separando i due stati

In [64]:
world_shp[world_shp['CNTRY_NAME']=='Sudan']

,OBJECTID,CNTRY_NAME,geometry
208,209,Sudan,"POLYGON ((35.82166 5.32861, 35.79874 5.26555, ..."


In [65]:
world_shp=world_shp[world_shp['CNTRY_NAME']!='Sudan']

In [66]:
world_shp[world_shp['CNTRY_NAME']=='Sudan']

,OBJECTID,CNTRY_NAME,geometry


In [67]:
from shapely.geometry import Point, LineString, Polygon

South_Sudan=Polygon([[33.963392794971185,9.464285229420625],[33.97498,8.68456],[33.82550000000015,8.37916],[33.29480000000012,8.35458],[32.95418,7.784970000000102],[33.568290000000104,7.71334],[34.0751,7.22595],[34.25032,6.82607],[34.70702,6.59422000000012],[35.298007118233095,5.506],[34.62019626785394,4.847122742082036],[34.005,4.249884947362147],[33.3900000000001,3.79],[32.68642,3.79232],[31.881450000000143,3.55827],[31.24556,3.7819],[30.83385,3.50917],[29.95349,4.1737],[29.71599531425602,4.600804755060153],[29.159078403446642,4.389267279473245],[28.696677687298802,4.455077215996994],[28.428993768027,4.287154649264608],[27.979977247842953,4.408413397637388],[27.374226108517632,5.233944403500175],[27.213409051225256,5.550953477394614],[26.465909458123292,5.946717434101856],[26.21341840994512,6.546603298362129],[25.796647983511264,6.97931590415817],[25.124130893664812,7.500085150579424],[25.114932488716875,7.825104071479245],[24.5673690121522,8.229187933785454],[23.886979580860665,8.619729712933065],[24.19406772118765,8.728696472403897],[24.53741516360202,8.91753756573172],[24.794925745412684,9.810240916008695],[25.069603699343986,10.273759963267992],[25.790633328413946,10.411098940233728],[25.962307049621018,10.136420986302424],[26.477328213242515,9.552730334198088],[26.752006167173818,9.466893473594496],[27.112520981708883,9.638567194801624],[27.833550610778783,9.60423245056029],[27.970889587744352,9.398223985111654],[28.966597170745782,9.398223985111654],[29.000931914987174,9.60423245056029],[29.515953078608614,9.793073543888056],[29.61895731133285,10.084918869940225],[29.99663949798855,10.290927335388687],[30.83784073190338,9.70723668328452],[31.35286189552488,9.810240916008695],[31.850715687025517,10.531270545078826],[32.400071594888345,11.080626452941488],[32.31423473428475,11.68148447716652],[32.073891524594785,11.973329803218519],[32.67474954881965,12.02483191958072],[32.743419037302544,12.248007757149992],[33.20693808456178,12.179338268667093],[33.086766479716744,11.441141267476496],[33.20693808456178,10.720111638406593],[33.72195924818311,10.325262079630193],[33.842130853028145,9.981914637215993],[33.82496348090751,9.484060845715362],[33.963392794971185,9.464285229420625]])
Sudan=Polygon([[33.963392794971185,9.464285229420625],[33.82496348090751,9.484060845715362],[33.842130853028145,9.981914637215993],[33.72195924818311,10.325262079630193],[33.20693808456178,10.720111638406593],[33.086766479716744,11.441141267476496],[33.20693808456178,12.179338268667093],[32.743419037302544,12.248007757149992],[32.67474954881965,12.02483191958072],[32.073891524594785,11.973329803218519],[32.31423473428475,11.68148447716652],[32.400071594888345,11.080626452941488],[31.850715687025517,10.531270545078826],[31.35286189552488,9.810240916008695],[30.83784073190338,9.70723668328452],[29.99663949798855,10.290927335388687],[29.61895731133285,10.084918869940225],[29.515953078608614,9.793073543888056],[29.000931914987174,9.60423245056029],[28.966597170745782,9.398223985111654],[27.970889587744352,9.398223985111654],[27.833550610778783,9.60423245056029],[27.112520981708883,9.638567194801624],[26.752006167173818,9.466893473594496],[26.477328213242515,9.552730334198088],[25.962307049621018,10.136420986302424],[25.790633328413946,10.411098940233728],[25.069603699343986,10.273759963267992],[24.794925745412684,9.810240916008695],[24.53741516360202,8.91753756573172],[24.19406772118765,8.728696472403897],[23.886979580860665,8.619729712933065],[23.805813429466752,8.666318874542526],[23.459012892355986,8.95428579348902],[23.394779087017298,9.26506785729225],[23.55724979014292,9.68121816653877],[23.554304233502194,10.08925527591532],[22.97754357269275,10.71446259199854],[22.864165480244253,11.142395127807617],[22.87622,11.384610000000123],[22.50869,11.67936],[22.49762,12.26024],[22.28801,12.64605],[21.93681,12.588180000000136],[22.03759,12.95546],[22.29658,13.37232],[22.18329,13.78648],[22.51202,14.09318],[22.30351,14.32682],[22.56795000000011,14.944290000000137],[23.024590000000103,15.68072],[23.886890000000108,15.61084],[23.837660000000138,19.580470000000105],[23.850000000000136,20],[25.00000000000011,20.00304],[25.00000000000011,22],[29.02,22],[32.9,22],[36.86623,22],[37.1887200000001,21.01885],[36.96941,20.83744000000013],[37.11470000000014,19.80796],[37.4817900000001,18.61409],[37.86276,18.36786],[38.410089959473225,17.99830739997031],[37.90400000000011,17.42754],[37.16747,17.263140000000135],[36.852530000000115,16.95655],[36.75389,16.29186],[36.32322,14.82249],[36.42951,14.42211],[36.27022,13.563330000000121],[35.86363,12.57828],[35.26049,12.08286],[34.83163000000013,11.318960000000118],[34.73115000000013,10.910170000000107],[34.25745,10.63009],[33.96162,9.58358],[33.963392794971185,9.464285229420625]])

In [68]:
world_shp.loc[251]=[252,'South Sudan',South_Sudan]
world_shp.loc[252]=[253,'Sudan',Sudan]

In [69]:
world_shp.loc[[251,252]]

,OBJECTID,CNTRY_NAME,geometry
251,252,South Sudan,"POLYGON ((33.96339 9.46429, 33.97498 8.68456, ..."
252,253,Sudan,"POLYGON ((33.96339 9.46429, 33.82496 9.48406, ..."


Si ri-ordina il dataset e lo si re-indicizza

In [70]:
world_shp=world_shp.sort_values('CNTRY_NAME')
world_shp=world_shp.reset_index(drop=True)

Rinominiamo gli stati africani in modo che questi coincidano col nome nel dataset wdi...

In [71]:
world_shp=world_shp.replace({'CNTRY_NAME' : 'Zaire'},
                                'Congo, Dem. Rep.')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Western Sahara'},
                                'W. Sahara')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Ivory Coast'},
                                "Cote d'Ivoire")
world_shp=world_shp.replace({'CNTRY_NAME' : 'Cape Verde'},
                                'Cabo Verde')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Congo'},
                                'Congo, Rep.')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Egypt'},
                                'Egypt, Arab Rep.')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Swaziland'},
                                'Eswatini')
world_shp=world_shp.replace({'CNTRY_NAME' : 'Tanzania, United Republic of'},
                                'Tanzania')

Consideriamo solo gli stati africani

In [72]:
africa_shp=world_shp[world_shp['CNTRY_NAME'].isin(africa)]
africa_shp=africa_shp.sort_values('CNTRY_NAME')
africa_shp = africa_shp.reset_index(drop=True)

In [73]:
len(africa)==len(africa_shp['CNTRY_NAME'])

True

In [74]:
africa_df = africa_shp[['CNTRY_NAME','geometry']]
africa_df2 = africa_shp[['CNTRY_NAME','geometry']]
africa_gdf = gpd.GeoDataFrame(africa_df)
africa_gdf2 = gpd.GeoDataFrame(africa_df2)

In [75]:
africa_gdf2['New Index']=range(55,110)
africa_gdf2.set_index('New Index', inplace=True)

In [76]:
for year in year_of_interest:
    africa_gdf[year]=round(total[year],1)
    africa_gdf[year+' color']=africa_gdf[year].apply(color_map_1)

In [77]:
for year in year_of_interest:
    africa_gdf2[year]=list(round(urban[year]-rural[year],1))
    africa_gdf2[year+' color']=africa_gdf2[year].apply(color_map_2)

In [78]:
africa_gdf_=[africa_gdf,africa_gdf2]

Creiamo due mappe centrate sull'Africa

In [82]:
slider_map_total=folium.Map(location=[6.292890, 21.420607], 
             minZoom=2.6, zoom_start=2.4, 
             min_lon=-45, max_lon=50,
             min_lat=-60, max_lat=100,
             max_bounds=True, 
             overlay=False,
             width='30%',
             height='80%',
             tiles=tiles[0])

slider_map_difference=folium.Map(location=[6.292890, 21.420607], 
             minZoom=2.6, zoom_start=2.4, 
             min_lon=-45, max_lon=50,
             min_lat=-60, max_lat=100,
             max_bounds=True, 
             overlay=False,
             width='30%',
             height='80%',
             tiles=tiles[0])

slider_map=[slider_map_total,slider_map_difference]

In [83]:
i1_='People using at least basic drinking water services (% of population)'
i2_='People using at least basic drinking water services (% of urban population - % of urban population)'
i_name_=[i1_,i2_]

In [84]:
colormaps=[cm.linear.YlGn_09.scale(0,100).to_step(5),
cm.LinearColormap(colors=['lightblue','mistyrose','red'], index=[-20,0,80],vmin=-20,vmax=80).to_step(5)]

In [85]:
for i in range(len(slider_map)):
    from folium.plugins import TimeSliderChoropleth

    _=TimeSliderChoropleth( 
        data=africa_gdf_[i].to_json(),
        styledict=style_dict[i],
        overlay=True,
        show=True
    ).add_to(slider_map[i])

    choropleth = folium.Choropleth(
        geo_data=africa_gdf.to_json(),
        name='choropleth',
        data=index_type[i],
        columns=['Country Name', '2020'],
        key_on='feature.properties.CNTRY_NAME',
        fill_color='YlGn',
        nan_fill_color='#808080',
        fill_opacity=0.0,
        line_opacity=0.0,
        highlight=True
        ).geojson.add_to(slider_map[i])
    
    #si aggiungono le etichette agli stati
    style_function = "font-size: 15px; font-weight: bold"
    choropleth.add_child(
    folium.features.GeoJsonTooltip(['CNTRY_NAME'], style=style_function, labels=False))

    slider_map[i].add_child(folium.Marker(
        location=[-40,85],
        icon = folium.features.CustomIcon(f'information-button.png',icon_size=(25, 26)),
        tooltip=f'info',
        draggable=False,
    ))
    
    folium.TileLayer(tiles[6],overlay=True,control=False,name=tiles[2]).add_to(slider_map[i])
    colormap=colormaps[i] 
    colormap.caption = i_name_[i]
    colormap.add_to(slider_map[i])

    slider_map[i].save(f'slider_map{i}.html')
    #folium.LayerControl(collapsed=True).add_to(slider_map[i])